# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

**Задача, которую необходимо решить:**<br>
Требуется построить модель, которая определяет рыночную стоимость автомобиля. 


**Заказчику важны:**<br>
- качество предсказания;
- скорость предсказания;
- время обучения.

**Цель проекта:**<br>
- Подготовить данные для дальнейшего исследования;
- Изучить представленные данные;
- Выполнить предобработку данных;
- Подготовить выборки для обучения моделей;
- Обучить разные модели, одна из которых — LightGBM, как минимум одна — не бустинг. Для каждой модели использовать разные гиперпараметры;
- Проанализировать время обучения, время предсказания;
- Проверку моделей провести по метрике RMSE. Значение метрики RMSE должно быть меньше 2500;
- Определить лучшую модель, опираясь на критерии заказчика.

**Входные данные:**<br>
Файл "autos.csv" - тренировочная выборка, которая содержит данные о технических характеристиках, комплектации и ценах других автомобилей.

**Описание данных:**<br>
*DateCrawled* — дата скачивания анкеты из базы<br>
*VehicleType* — тип автомобильного кузова<br>
*RegistrationYear* — год регистрации автомобиля<br>
*Gearbox* — тип коробки передач<br>
*Power* — мощность (л. с.)<br>
*Model* — модель автомобиля<br>
*Kilometer* — пробег (км)<br>
*RegistrationMonth* — месяц регистрации автомобиля<br>
*FuelType* — тип топлива<br>
*Brand* — марка автомобиля<br>
*Repaired* — была машина в ремонте или нет<br>
*DateCreated* — дата создания анкеты<br>
*NumberOfPictures* — количество фотографий автомобиля<br>
*PostalCode* — почтовый индекс владельца анкеты (пользователя)<br>
*LastSeen* — дата последней активности пользователя<br>

**Целевой признак:**<br>
*Price* — цена (евро)<br>

**Методология и инструменты**<br><br>
**Методы:**<br>
- Сбор данных: таблица, полученная от заказчика: "autos.csv".
- Очистка данных: данные будут очищены от дубликатов и пропущенных значений, типы данных приведены в соответствие.
- Анализ данных: исследовательский анализ данных из таблицы.
- Корреляционный анализ данных.
- Обучение моделей: обучение моделей с разными гиперпараметрами.
- Проверка качества работы модели на тестовой выборке.

**Инструменты:**<br>
- Python: Основной язык программирования для анализа данных.
- Pandas: Библиотека для обработки и анализа данных.
- Matplotlib и Seaborn: Библиотеки для визуализации данных.
- Sklearn: Библиотека для машинного обучения.

**Дополнительные методы и инструменты:**<br>
**Jupyter Notebook:** Для интерактивного анализа данных, создания отчетов, обучение и проверка моделей.

In [ ]:
import re
import warnings
from warnings import simplefilter

!pip install scikit-learn==1.1.0 -q
!pip install matplotlib==3.5.1 -q
!pip install tqdm
!pip install phik
!pip install shap
!pip install lightgbm
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import phik

from pylab import rcParams
from scipy import stats as st
from plotly.subplots import make_subplots

from phik.report import plot_correlation_matrix

from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV

# загружаем класс pipeline
from sklearn.pipeline import Pipeline

# загружаем классы для подготовки данных
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

# загружаем класс для работы с пропусками
from sklearn.impute import SimpleImputer

# загружаем функцию для работы с метриками
from sklearn.metrics import make_scorer, roc_auc_score, mean_squared_error

# загружаем необходимые модели
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

from time import time

# настройки pandas для отображения данных
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

# игнорирование предупреждения
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# подготовка констант:
RANDOM_STATE = 42
TEST_SIZE = 0.25

In [ ]:
# цвета для графиков:
colors = (['#063751','#08527A',
           '#0B6DA2','#0E89CB',
           '#15A3EF','#3EB3F2',
           '#66C3F4','#8FD3F7',
           '#B7E3FA','#B7E3FA',
           '#8FD3F7','#66C3F4',
           '#3EB3F2','#15A3EF',
           '#0E89CB','#0B6DA2',
           '#08527A','#063751'])

In [ ]:
# функция data_info отображает основную информацию об имеющихся данных из указанного файла
def data_info(data):
    try:
        display('Общая информация')
        data.info()
        display('Первые пять строк данных:')
        display(data.head())
        display('Описание данных:')
        display(data.describe())
        display('Количество пропусков')
        display(data.isna().sum())
        display('Кол-во явных дубликатов:')
        display(data.duplicated().sum())
    except:
        return 'Проверьте параметры'
    
# функция для перевода названия столбцов из CamelCase в snake_case
def to_snake_case(columns):
    new_cols = []
    for name in columns:
        name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
        name = re.sub('__([A-Z])', r'_\1', name)
        name = re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()
        new_cols.append(name)
    return new_cols    
    
# функция диаграммы размаха
def span_diagramm(data, param1, span_name):
    try:
        fig = px.box(data, 
             x=data[param1], 
             color_discrete_sequence=[colors[2]],
             title=span_name)
        fig.update_layout(plot_bgcolor='AliceBlue',
                  margin={"r": 0, "t": 50, "l": 0, "b": 50})
        fig.show()       
    except:
        return 'Проверьте параметры'

## Загрузка данных

### Открытие файла с данными

In [ ]:
# загрузка данных из csv-файла "autos.csv" в датафрейм df_autos
try:
    df_autos = pd.read_csv('autos.csv', parse_dates=['DateCrawled', 'DateCreated', 'LastSeen'])
except:
    df_autos = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv', parse_dates=['DateCrawled', 'DateCreated', 'LastSeen'])

### Изучение общей информации

In [ ]:
# общая информация для df_autos
data_info(df_autos)

В таблице представлены данные о технических характеристиках, комплектации и ценах автомобилей.

Записей об автомобилях в таблице 354 369, они представляют из себя:

- дата скачивания анкеты из базы
- цена (евро)
- тип автомобильного кузова
- год регистрации автомобиля
- тип коробки передач
- мощность (л. с.)
- модель автомобиля
- пробег (км)
- месяц регистрации автомобиля
- тип топлива
- марка автомобиля
- была машина в ремонте или нет
- дата создания анкеты
- количество фотографий автомобиля
- почтовый индекс владельца анкеты (пользователя)
- дата последней активности пользователя

Количество явных дубликатов 4.<br><br>

При первичном ознакомлении с данными в таблице обнаружены пропуски: 
- VehicleType 37490, 
- Gearbox 19833, 
- Model 19705, 
- FuelType 32895, 
- Repaired 71154.<br><br>

Предварительно обнаружены аномальные значения:
- Price - минимальная цена 0. Такого быть не должно.
- RegistrationYear - машина явно не может быть зарегистрирована в 9999 году, как и в 1000
- Power - 20 000 лошадиных сил. Слишком много.
- NumberOfPictures - предполагаем, картинок не было в данных, поскольку наблюдаем только нули, либо это техническая неполадка.<br><br>
Типы данных соответствуют требуемым.

##  Предобработка данных

In [ ]:
# перевод названий столбцов в snake_case
df_autos.columns = to_snake_case(df_autos.columns)
df_autos.head()

In [ ]:
# пайплайн для замены пропущенных значений
#imputer_pipe = Pipeline(
#    [
#        (
#            'SimpleImputer',
#            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#        ),
#    ]
#)

# обработка пропущенных значений при помощи пайплайна
#nan_cols = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'repaired']
#df_autos[nan_cols] = imputer_pipe.fit_transform(df_autos[nan_cols])

In [ ]:
# проверка
#print(f'Количество пропущенных значений после замены: \n {df_autos.isna().sum()}')
#print('')
#print(f'Проверка исправности данных: {df_autos.info()}')

In [ ]:
# удаление дубликатов 
print(f'Кол-во явных дубликатов до удаления: {df_autos.duplicated().sum()}')
df_autos.drop_duplicates(inplace=True)
print(f'Кол-во явных дубликатов после удаления: {df_autos.duplicated().sum()}')

##  Исследовательский анализ данных

In [ ]:
df_autos['price'].describe()

In [ ]:
span_diagramm(df_autos, 'price', 'Диаграмма размаха по цене')

In [ ]:
# гистограмма по price
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot()
 
ax.hist(df_autos['price'], 30)
ax.grid()

plt.title('Распределение признака price')
plt.xlabel('Цена')
plt.ylabel('Автомобили')
plt.show()

In [ ]:
# автомобили стоимостью более 14 400 евро
len(df_autos.query('price > 14400'))

In [ ]:
print(f"Доля автомобилей стоимостью более 14 400 евро: {len(df_autos.query('price > 14400')) / len(df_autos):%} ")

In [ ]:
len(df_autos.query('price < 1000'))

In [ ]:
# удаление автомобилей стоимостью ниже 1000 евро
print(f'Размерность датасета ДО удаления автомобилей стоимостью до 1000 евро: {df_autos.shape}')
df_autos = df_autos.drop(df_autos[df_autos['price'] < 1000].index)
print(f'Размерность датасета ПОСЛЕ удаления автомобилей стоимостью до 1000 евро: {df_autos.shape}')

In [ ]:
span_diagramm(df_autos, 'price', 'Диаграмма размаха по цене')

In [ ]:
# гистограмма по price
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot()
 
ax.hist(df_autos['price'], 30)
ax.grid()

plt.title('Распределение признака price')
plt.xlabel('Цена')
plt.ylabel('Автомобили')
plt.show()

У таргета наблюдаем распределение Пуассона.<br>
Крупных выбросов не так много: они составляют 5.35% от всех данных. Учтем это при обучении моделей.<br>
Избавились от автомобилей стоимостью ниеж 1000 евро в записях - удалено 83 326 строк.

In [ ]:
# частота встречаемости типов автомобильного кузова
df_autos.groupby('vehicle_type')['date_crawled'].count()

In [ ]:
fig = px.pie((df_autos
             .groupby('vehicle_type')['date_crawled']
             .count()
             .reset_index()
             .rename(columns={
                 'vehicle_type': 'Тип кузова', 
                 'date_crawled': 'Автомобили'}))
             .sort_values(by='Автомобили', ascending=False),
             
             values='Автомобили', 
             names='Тип кузова', 
             color_discrete_sequence=[colors[0], colors[7]],
             title='Распределение долей автомобилей по типу кузова', 
             height=500)

fig.update_traces(textposition='inside', textinfo='percent', hovertemplate='')
fig.update_layout(showlegend=True, legend_title='Тип кузова',
                  plot_bgcolor='AliceBlue',
                  margin={"r": 0, "t": 50, "l": 0, "b": 50})

fig.show()

Распределение долей автомобилей по типу кузова: 
- sedan - 29.6 %;
- wagon - 21.2 %;
- small - 20.5 %;
- bus - 10.4 %;
- convertible - 7.47 %;
- coupe - 5.35 %;
- suv - 4.57 %;
- other - 0.94 %.

In [ ]:
df_autos['registration_year'].describe()

In [ ]:
span_diagramm(df_autos, 'registration_year', 'Диаграмма размаха по году регистрации автомобиля')

In [ ]:
# гистограмма по registration_year
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot()
 
ax.hist(df_autos['registration_year'], 30)
ax.grid()

plt.title('Распределение признака registration_year')
plt.xlabel('Год регистрации')
plt.ylabel('Автомобили')
plt.show()

Верхняя граница допустимых значений для года регистрации можно считать 2016, так как это год создания анкеты.

In [ ]:
print(f"Доля авто с годом регистрации больше 2016: \
    {len(df_autos.query('registration_year > 2016')) / len(df_autos):%} ")

print(f"Доля авто с годом регистрации раньше 1900: \
    {len(df_autos.query('registration_year < 1900')) / len(df_autos):%} ")

Год регистрации автомобиля физически не может быть больше года создания анкеты (2016) 3.92% или меньше 1900 0.006%. 
Данные придется удалить, так как они довольно сильно отражаются на распределении, однако не занимают существенной доли среди всех записей. 
Автозаполнить год регистрации автомобиля также не является грамотным решением на самый популярный год.

In [ ]:
# удаление записей с годом регистрации авто позднее 2016 и ранее 1900
print(f'Размер датасета ДО удаления строк с аномальными значениями: {df_autos.shape}')
print(
    f'Удалено {len(df_autos.query("registration_year > 2016")) + len(df_autos.query("registration_year < 1900"))} строк')

df_autos = df_autos.drop(df_autos[df_autos['registration_year'] < 1900].index)
df_autos = df_autos.drop(df_autos[df_autos['registration_year'] > 2016].index)

print(f'Размер датасета ПОСЛЕ удаления строк с аномальными значениями: {df_autos.shape}')

In [ ]:
span_diagramm(df_autos, 'registration_year', 'Диаграмма размаха по году регистрации автомобиля')

In [ ]:
# гистограмма по registration_year
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot()
 
ax.hist(df_autos['registration_year'], 30)
ax.grid()

plt.title('Распределение признака registration_year')
plt.xlabel('Год регистрации')
plt.ylabel('Автомобили')
plt.show()

Распределение признака registration_year близко к нормальному.

In [ ]:
# частота встречаемости типов коробки передач
df_autos.groupby('gearbox')['date_crawled'].count()

In [ ]:
fig = px.pie((df_autos
             .groupby('gearbox')['date_crawled']
             .count()
             .reset_index()
             .rename(columns={
                 'gearbox': 'Тип коробки передач', 
                 'date_crawled': 'Автомобили'}))
             .sort_values(by='Автомобили', ascending=False),
             
             values='Автомобили', 
             names='Тип коробки передач', 
             color_discrete_sequence=[colors[0], colors[7]],
             title='Распределение долей автомобилей по типу коробки передач', 
             height=500)

fig.update_traces(textposition='inside', textinfo='percent', hovertemplate='')
fig.update_layout(showlegend=True, legend_title='Тип коробки передач',
                  plot_bgcolor='AliceBlue',
                  margin={"r": 0, "t": 50, "l": 0, "b": 50})

fig.show()

Распределение долей автомобилей по типу коробки передач: 
- manual - 77.1%; 
- auto - 22.9%.

In [ ]:
# частота встречаемости моделей автомобиля
df_autos['model'].value_counts()

Среди моделей автомобилей лидирует **golf.** 

In [ ]:
df_autos['power'].describe()

In [ ]:
span_diagramm(df_autos, 'power', 'Диаграмма размаха по мощности')

In [ ]:
# гистограмма по power
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot()
 
ax.hist(df_autos['power'], 30)
ax.grid()

plt.title('Распределение признака power')
plt.xlabel('Мощность')
plt.ylabel('Автомобили')
plt.show()

In [ ]:
print(f"Доля авто с количеством лошадиных сил == 0: \
    {len(df_autos.query('power == 0')) / len(df_autos):%} ")

print(f"Доля авто с количеством лошадиных сил более 1000: \
    {len(df_autos.query('power > 1000')) / len(df_autos):%} ")

Мощность 0 лошадиных сил невозможна.<br>
Подобных записей в данных весьма много (7.15%): предлагаю заменить нули на медианное значение количества лошадиных сил среди подобных брендов и моделей автомобиля.<br>
В том числе являются странными записи, в которых в автомобилях более 1000 л.с. Таких записей меньше - 0.07%, однако предлагаю заменить подобные значения по тому же принципу.

In [ ]:
# замена нулей и нужных значений в столбце 'power' на NaN, чтобы они не учитывались при вычислении медианы
df_autos['power'].replace(0, pd.NA, inplace=True)
df_autos.loc[(df_autos['power'] > 1000), 'power'] = pd.NA

df_autos['power'] = pd.to_numeric(df_autos['power'], errors='coerce')

# медиана для каждой уникальной пары 'brand' и 'model'
median_power = df_autos.groupby(['brand', 'model'])['power'].median().reset_index()

# объединение данных по медианам с основным датасетом по 'brand' и 'model'
df_autos = pd.merge(df_autos, median_power, on=['brand', 'model'], how='left', suffixes=('', '_median'))

# проверка объединения
print('Датасет ДО изменения, с дополнительным столбцом: \n')
display(df_autos.head())
display(df_autos.shape)
display(df_autos.info())

# замена значений 'power' на медианные значения, если исходное значение было 0 или больше 1000
df_autos['power'] = df_autos.apply(lambda row: row['power_median'] if pd.isna(row['power']) else row['power'],
                                       axis=1)

# удаление вспомогательной колонки с медианными значениями
df_autos.drop(columns=['power_median'], inplace=True)

print(' ')
print('Датасет ПОСЛЕ изменения: \n')
display(df_autos.head())
display(df_autos.shape)
display(df_autos.info())

In [ ]:
print(f'Проверка на пропущенные значения: {df_autos.isna().sum()}')

In [ ]:
df_autos[df_autos['power'].isna()]

In [ ]:
# замена пропущенных значений на медианные по столбцу
all_median = df_autos['power'].median()
df_autos['power'].fillna(all_median, inplace=True)
df_autos['power'].isna().sum()

In [ ]:
df_autos['kilometer'].describe()

In [ ]:
span_diagramm(df_autos, 'kilometer', 'Диаграмма размаха по пробегу')

In [ ]:
# гистограмма по kilometer
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot()
 
ax.hist(df_autos['kilometer'], 12)
ax.grid()

plt.title('Распределение признака kilometer')
plt.xlabel('Пробег')
plt.ylabel('Автомобили')
plt.show()

In [ ]:
print(f"Доля авто  с пробегом менее 30 000 км: \
    {len(df_autos.query('kilometer < 30000')) / len(df_autos):%} ")

На графике наблюдаются выбросы: это записи авто, имеющих пробег менее 30 000 км. Такие записи составляют ~ 2.6% от общего числа данных, в связи с чем предлагаю оставить данные. Показатели пробега менее 30 000 км не будем считать аномальными, так как это стандартный диапазон пробега - это нормальное явление.

In [ ]:
df_autos['registration_month'].describe()

In [ ]:
span_diagramm(df_autos, 'registration_month', 'Диаграмма размаха по месяцам регистрации автомобиля')

In [ ]:
# гистограмма по registration_month
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot()
 
ax.hist(df_autos['registration_month'], 30)
ax.grid()

plt.title('Распределение признака registration_month')
plt.xlabel('Месяц регистрации')
plt.ylabel('Автомобили')
plt.show()

Распределение данных по месяцу регистрации автомобилей похоже на среднестатистические данные и распределены практически равномерно

In [ ]:
# частота встречаемости типов топлива
df_autos.groupby('fuel_type')['date_crawled'].count()

In [ ]:
fig = px.pie((df_autos
             .groupby('fuel_type')['date_crawled']
             .count()
             .reset_index()
             .rename(columns={
                 'fuel_type': 'Тип топлива', 
                 'date_crawled': 'Автомобили'}))
             .sort_values(by='Автомобили', ascending=False),
             
             values='Автомобили', 
             names='Тип топлива', 
             color_discrete_sequence=[colors[0], colors[7]],
             title='Распределение долей автомобилей по типу топлива', 
             height=500)

fig.update_traces(textposition='inside', textinfo='percent', hovertemplate='')
fig.update_layout(showlegend=True, legend_title='Тип топлива',
                  plot_bgcolor='AliceBlue',
                  margin={"r": 0, "t": 50, "l": 0, "b": 50})

fig.show()

Распределение долей автомобилей по типу топлива: 
- petrol - 61.5%; 
- gasoline - 36.4%; 
- lpg - 1.77%; 
- cng - 0.197%; 
- hybrid - 0.08%; 
- other - 0.036%;
- electric - 0.03%.

In [ ]:
# частота встречаемости марок автомобилей
df_autos['brand'].value_counts()

Среди марок автомобилей лидирует **volkswagen.** <br>
В пятерку лидеров также вошли марки: **opel, bmw, mercedes_benz, audi.**

In [ ]:
# частота встречаемости ремонтируемости автомобиля
df_autos.groupby('repaired')['date_crawled'].count()

In [ ]:
fig = px.pie((df_autos
             .groupby('repaired')['date_crawled']
             .count()
             .reset_index()
             .rename(columns={
                 'repaired': 'Ремонтировался ли автомобиль', 
                 'date_crawled': 'Автомобили'}))
             .sort_values(by='Автомобили', ascending=False),
             
             values='Автомобили', 
             names='Ремонтировался ли автомобиль', 
             color_discrete_sequence=[colors[0], colors[7]],
             title='Распределение долей автомобилей по признаку ремонтировался ли автомобиль', 
             height=500)

fig.update_traces(textposition='inside', textinfo='percent', hovertemplate='')
fig.update_layout(showlegend=True, legend_title='Ремонтировался ли автомобиль',
                  plot_bgcolor='AliceBlue',
                  margin={"r": 0, "t": 50, "l": 0, "b": 50})

fig.show()

Распределение долей автомобилей по признаку ремонтировался ли автомобиль:
- no - 92.6%;
- yes - 7.37%.

In [ ]:
df_autos['number_of_pictures'].describe()

In [ ]:
span_diagramm(df_autos, 'number_of_pictures', 'Диаграмма размаха по количеству фотографий автомобиля')

In [ ]:
# гистограмма по number_of_pictures
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot()
 
ax.hist(df_autos['number_of_pictures'], 30)
ax.grid()

plt.title('Распределение признака number_of_pictures')
plt.xlabel('Количество фотографий')
plt.ylabel('Автомобили')
plt.show()

Как мы выяснили еще на этапе загрузки данных - картинки не прогрузились ни для одной записи. Либо это крупный технический сбой, либо картинок в целом нет у отобранных объявлений.

**Вывод:**<br>
- Избавились от автомобилей стоимостью ниже 1000 евро в записях - удалено 83 326 строк;
- Убрали 10 643 строк из данных, связанных с годом регистрации автомобиля: машины, зарегистрированные до 1900 и после 2016;
- Обнаружили строки с мощностью автомобиля 0 лошадиных сил. Подобных записей достаточно много. Заменили нули на медианное значение количества лошадиных сил среди подобных брендов и моделей автомобиля.
- Обнаружили странные записи, в которых в автомобилях более 1000 лошадиных сил. Скорее всего это техническая ошибка сервиса. Заменили отобранные значения по тому же принципу, что и с 0 лошадиными силами.
- Среди типов автомобильных кузовов лидирует "седан" - 29.6%;
- Лидирующий тип коробки передач ручной - 77.1%;
- Среди моделей автомобилей лидирует golf;
- Среди брендов лидер volkswagen;
- Лидирует тип топлива бензин - 61.5%;
- Большинство машин 92.6% не было в ремонте.

##  Корреляционный анализ

In [ ]:
# корреляционный анализ
data_autos = df_autos.copy()
data_autos = data_autos.drop(['date_crawled', 'date_created', 'last_seen', 'number_of_pictures'], axis=1)

In [ ]:
phik_matrix_autos = data_autos.phik_matrix(interval_cols=['price', 'power', 'kilometer', 'postal_code'])

In [ ]:
# визуализация корреляционной матрицы 
mask = np.triu(np.ones_like(phik_matrix_autos, dtype=bool))

plt.figure(figsize=(12, 8))
sns.heatmap(phik_matrix_autos, annot=True, fmt=".2f", cmap="coolwarm", mask=mask)
plt.title("Корреляционная матрица")
plt.show()

Корреляция между Ценой и другими признаками:

- год регистрации: корреляция умеренно-положительная. Коэффициент взаимосвязи: 0.64. Это логично, так как, чем свежее автомобиль, тем он более востребован, а значит и дороже;
- модель: закономерно, корреляция умеренно-положительная. Одни модели ценятся больше у автовладельцев, другие меньше. Коэффициент взаимосвязи: 0.54;

Нецелевые признаки:
- умеренная линейная зависимость марки машины с типом кузова, что тоже довольно логично, поскольку наполнение авто напрямую зависит от бренда.
- высокий коэффициент корреляции между model и vehicle_type - 0.92. Тоже весьма понятно, что разные модели имеют место на существование при разных типажах кузова. 
- сильнейшая связь модели от бренда, что вполне логично, модель без бренда не может существовать. Коэффициент корреляции 1. 
- почтовый индекс владельца анкеты: очень слабо коррелирует с другими признаками, в связи с чем можем от него смело избавиться.

## Подготовка данных для обучения

Избавимся от следующих столбцов, которые не принесут пользы для обучения:<br>
- number_of_pictures - в признаке только нули, это скорее техническая ошибка;
- date_crawled, date_created, last_seen - даты, которые также не понесут в себе особой пользы в обучении;
- postal_code - слабо связанный со всеми признаками. Не несет практической значимости для предсказания целевого признака.

In [ ]:
df_autos_ml = df_autos.drop(['number_of_pictures',
                                 'date_crawled',
                                 'date_created',
                                 'last_seen',
                                 'postal_code',
                                 'registration_month'
                                 ], axis=1)

df_autos_ml.head()

In [ ]:
# проверка на пропущенные значения и дубликаты
print(f'Пропущенные значения: {df_autos_ml.isna().sum()}')
print(f'Количество дубликатов: {df_autos_ml.duplicated().sum()}')
print(f'Доля дубликатов от всех данных: {df_autos_ml.duplicated().sum() / len(df_autos_ml):%}')

In [ ]:
# избавляемся от дубликатов
df_autos_ml.drop_duplicates(inplace=True)

# проверка
print(f'Количество дубликатов после удаления: {df_autos_ml.duplicated().sum()}')

In [ ]:
# разбиваем фичи и таргет
X = df_autos_ml.drop('price', axis=1)
y = df_autos_ml['price']

# разбиваем выборку на тренировочную и тестовую
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE)

print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_train: {y_train.shape}')
print(f'y_test: {y_test.shape}')

### Пайплайн предобработки

In [ ]:
# создаём списки с названиями признаков
ohe_columns = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired']
ord_columns = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired']
num_columns = ['registration_year', 'power', 'kilometer']

ohe_pipe = Pipeline([
    ('simpleImputer_before_ohe', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('ohe', OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)),
    ('simpleImputer_after_ohe', SimpleImputer(missing_values=np.nan, strategy='most_frequent'))
])

ord_pipe = Pipeline([
    ('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)),
    ('simpleImputer_after_ord', SimpleImputer(missing_values=np.nan, strategy='most_frequent'))
])

# создаём общий пайплайн для подготовки данных линейных моделей
data_preprocessor_lin = ColumnTransformer(
    [
        ('ohe', ohe_pipe, ohe_columns),
        ('num', StandardScaler(), num_columns)
    ],
    remainder='passthrough'
)

# создаём общий пайплайн для подготовки данных деревьев
data_preprocessor_tree = ColumnTransformer(
    [
        ('ord', ord_pipe, ord_columns),
        ('num', StandardScaler(), num_columns)
    ],
    remainder='passthrough'
)

# Cоздаём общий пайплайн для подготовки данных деревьев
data_preprocessor_tree = ColumnTransformer(
    [
        ('ord', ord_pipe, ord_columns),
        ('num', StandardScaler(), num_columns)
    ],
    remainder='passthrough'
)

# создаём итоговый пайплайн: подготовка данных и базовая модель для линейных моделей
pipe_final_lin = Pipeline([
    ('preprocessor', data_preprocessor_lin),
    ('models', LinearRegression())
])

# создаём итоговый пайплайн: подготовка данных и базовая модель для деревьев
pipe_final_tree = Pipeline([
    ('preprocessor', data_preprocessor_tree),
    ('models', RandomForestRegressor(random_state=RANDOM_STATE))
])

X_train_processed_lin = data_preprocessor_lin.fit_transform(X_train)
X_test_processed_lin = data_preprocessor_lin.transform(X_test)

X_train_processed_tree = data_preprocessor_tree.fit_transform(X_train)
X_test_processed_tree = data_preprocessor_tree.transform(X_test)

## Обучение моделей

### LinearRegression

In [ ]:
#%%time
start = time()

lin_reg = pipe_final_lin 
lin_reg.fit(X_train, y_train)

end = time()
linear_time = (end-start)/60

print('Время обучения модели: ', linear_time)

In [ ]:
scores = cross_val_score(
    lin_reg,
    X_train,
    y_train,
    cv=5,
    n_jobs=-1,
    scoring='neg_root_mean_squared_error'
)

cv_score_lin_reg = sum(scores) / len(scores)
print('RMSE на кросс-валидации :', (-1) * cv_score_lin_reg)

In [ ]:
#%%time
start = time()

y_pred_lin_reg = lin_reg.predict(X_train)
#print('RMSE на тренировочной выборке:', mean_squared_error(y_train, y_pred_lin_reg, squared=False))

end = time()
linear_predict_time = (end-start)/60

print('Время предсказания модели:', linear_predict_time)

**Качество предсказания LinearRegression:**<br>
- **RMSE на кросс-валидации:** 2846.455289893139
- **Время обучения модели:** 0.35 min
- **Время предсказания модели:** 0.03 min

### LGBMRegressor

In [ ]:
# подберем лучшие гиперпараметры

lgbm_pipe = Pipeline([
    ('preprocessor', data_preprocessor_tree),
    ('models', LGBMRegressor(random_state=RANDOM_STATE, verbose=-1))
])

lgbm_params = {
    'models': [LGBMRegressor(random_state=RANDOM_STATE, verbose=-1)],
    'models__n_estimators': range(50, 100, 10),
    'models__max_depth': range(3, 8),
    'models__num_leaves': range(5, 50, 5)
}

lgbm_reg = RandomizedSearchCV(
    lgbm_pipe,
    lgbm_params,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    n_iter=10,
    cv=5,
    verbose=5,
    random_state=RANDOM_STATE,
)

lgbm_reg.fit(X_train, y_train)

# лучшие гиперпараметры
print('Лучшая модель: \n', lgbm_reg.best_estimator_)
# лучшая метрика качества
print('RMSE на кросс-валидации', (-1) * lgbm_reg.best_score_)

In [ ]:
#%%time
start = time()

lgbm_reg.best_estimator_.fit(X_train, y_train)

end = time()
lgbm_time = (end-start)/60

print('Время обучения модели: ', lgbm_time)

In [ ]:
#%%time
start = time()

y_pred_lgbm_reg = lgbm_reg.best_estimator_.predict(X_train)
#print('RMSE на тренировочной выборке:', mean_squared_error(y_train, y_pred_lgbm_reg, squared=False))

end = time()
lgbm_predict_time = (end-start)/60

print('Время предсказания модели:', lgbm_predict_time)

**Качество предсказания LGBMRegressor:**<br>
- **RMSE на кросс-валидации:** 1850.6183606205384
- **Время обучения модели:** 0.06 min
- **Время предсказания модели:** 0.0 2min

### CatBoostRegressor

In [ ]:
# пайплайн для замены пропущенных значений
imputer_pipe = Pipeline(
    [
        (
            'SimpleImputer',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
    ]
)

# обработка пропущенных значений при помощи пайплайна
nan_cols = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'repaired']
X_train[nan_cols] = imputer_pipe.fit_transform(X_train[nan_cols])

In [ ]:
#%%time
start = time()

catboost_model = CatBoostRegressor(random_state=RANDOM_STATE, cat_features=ohe_columns, verbose=False)
catboost_model.fit(X_train, y_train)

end = time()
catboost_time = (end-start)/60

print('Время обучения модели: ', catboost_time)

In [ ]:
catboost_scores = cross_val_score(
    catboost_model,
    X_train,
    y_train,
    cv=5,
    n_jobs=-1,
    scoring='neg_root_mean_squared_error'
)

cv_score_catboost = sum(catboost_scores) / len(catboost_scores)
print('RMSE на кросс-валидации :', (-1) * cv_score_catboost)

In [ ]:
#%%time
start = time()

y_pred_catboost = catboost_model.predict(X_train)
#print('RMSE на тренировочной выборке:', mean_squared_error(y_train, y_pred_catboost, squared=False))

end = time()
catboost_predict_time = (end-start)/60

print('Время предсказания модели:', catboost_predict_time)

**Качество предсказания CatBoostRegressor:**<br>
- **RMSE на кросс-валидации:** 1760.2421775419946
- **Время обучения модели:** 1.96 min
- **Время предсказания модели:** 0.02 min

## Анализ моделей

In [ ]:
results = {
    'model': ['LinearRegression', 'LGBMRegressor', 'CatBoostRegressor'],
    'RMSE_cross_val': [(-1) * cv_score_lin_reg, (-1) * lgbm_reg.best_score_, (-1) * cv_score_catboost],
    'train_time': [linear_time, lgbm_time, catboost_time],
    'predict_time': [linear_predict_time, lgbm_predict_time, catboost_predict_time]
}

all_models = pd.DataFrame(results)

all_models.sort_values(by='RMSE_cross_val')

Лучше всего с задачей справилась модель с дефолтными настройками - CatBoostRegressor. Она показала лучший RMSE на кросс-валидации и быстрее остальных моделей предсказывает значения.<br>
Выбираем CatBoostRegressor

In [ ]:
%%time

X_test[nan_cols] = imputer_pipe.fit_transform(X_test[nan_cols])
y_pred = catboost_model.predict(X_test)
print('RMSE лучшей модели тестовой выборки:', mean_squared_error(y_test, y_pred, squared=False))

## Общий вывод

В ходе работы над исследованием была построена модель машинного обучения для определения рыночной стоимости автомобиля, соответствующая критериям заказчика.<br>

Исследование было разделено на несколько этапов:

1. **Загрузка и подготовка данных**

В таблице представлены данные о технических характеристиках, комплектации и ценах автомобилей. Всего в датасете 354 369 записей.<br>
Обнаружили пропущенные значения:<br>
VehicleType - 37 490<br>
Gearbox - 19 833<br>
Model - 19 705<br>
FuelType - 32 895<br>
Repaired - 71 154<br>
Обнаружили 4 явных дубликата<br>
Предварительно определили аномальные значения:<br>
Price - минимальная цена 0. Такого быть не должно.<br>
RegistrationYear - машина явно не может быть зарегистрирована в 9999 году, как и в 1000<br>
Power - 20 000 лошадиных сил. Слишком много.<br>
NumberOfPictures - предполагаем, картинок не было в данных, поскольку наблюдаем только нули, либо это техническая неполадка.<br>
С типами данных все в порядке.


2. **Предобработка данных**

Привели названия столбцов к snake_case для удобства.<br>
Избавились от 5-ти явных дубликатов в данных.

3. **Исследовательский анализ данных**

Избавились от автомобилей стоимостью ниже 1000 евро в записях - удалено 83 326 строк;<br>
Убрали 10 643 строк из данных, связанных с годом регистрации автомобиля: машины, зарегистрированные до 1900 и после 2016;<br>
Обнаружили строки с мощностью автомобиля 0 лошадиных сил. Подобных записей достаточно много. Заменили нули на медианное значение количества лошадиных сил среди подобных брендов и моделей автомобиля;<br>
Обнаружили странные записи, в которых в автомобилях более 1000 лошадиных сил. Скорее всего это техническая ошибка сервиса. Заменили отобранные значения по тому же принципу, что и с 0 лошадиными силами;<br>
Среди типов автомобильных кузовов лидирует "седан" - 29.6%;<br>
Лидирующий тип коробки передач ручной - 77.1%;<br>
Среди моделей автомобилей лидирует golf;<br>
Среди брендов лидер volkswagen;<br>
Лидирует тип топлива бензин - 61.5%;<br>
Большинство машин 92.6% не было в ремонте.<br>

4. **Корреляционный анализ данных**

Целевой признак коррелирует умеренно-положительно (0.5 < y < 0.8) с несколькими признаками:<br>
год регистрации: корреляция умеренно-положительная. Коэффициент взаимосвязи: 0.64. Это логично, так как, чем свежее автомобиль, тем он более востребован, а значит и дороже;<br>
модель: закономерно, корреляция умеренно-положительная. Одни модели ценятся больше у автовладельцев, другие меньше. Коэффициент взаимосвязи: 0.54.

5. **Обучение моделей**

Подготовили выборки, убрав неважные признаки;<br>
Создали пайплайн предобработки;<br>
Подобрали гиперпараметры некоторых моделей и обучили + замерили метрики на кросс-валидации и трейне.

6. **Анализ моделей**

Лучше всего с задачей справилась модель с классическими гиперпараметрами - CatBoostRegressor. Она показала лучший RMSE на кросс-валидации и быстрее остальных моделей предсказывает значения.<br>
RMSE на кросс-валидации: 1760.2421775419946<br>
RMSE на тестовой выборке: 1732.5431702466208<br>
Время обучения модели: 1.96 min<br>
Время предсказания модели (на тестовой выборке): 424 ms
<br>

Конечная модель отвечает всем критериям заказчика:<br>
Предсказание качественно и метрика RMSE < 2500.